CUDA 可用streams 或Graph工作。

stream 为有向无环图。

graph node is any **asynchronous** CUDA operation.

Graph is sequence of *operations*, connected by *dependencies*.


Operations are one of:
    Kernel Launch
    CPU Function Call
    Memcopy/Memset
    Memory Alloc/Free
    Sub-Graph
    

好处：
节省小且多的stream启动开销（microseconds）；节省cpu 。


CUDA graph has Three-stage execution model

Define:
    Single Graph "Template".

    Created in host code, or loaded from disk,
    or built up from libraries.

Instantiate:
    Multiple "Executable Graphs"

    Snapshot of template .
    Sets up & initializes GPU execution structures.
    (create once, run many times )

Execute:
    Running in CUDA Streams.
    Concurrency in graph *is not* limited by stream



Modifying Graphs in-place

Stream Launch:

Graph Update:

Graph Re-launch


2 ways to create CUDA graphs

1. Stream Capture

    Records operations without actually launching a kernel

    Library must call an API to tell if kernels are being captured instead of launched.
    #### Capture operation falls
    . Problem if libray calls cudaStreamSynchronize() or any other synchronous operation.
    . Capture is not launching anything so synchronize cannot wait for anything.



In [ ]:
cudaStreamBeginCapture(&stream1);

//build stream work as usual 
A<<< ..., stream1>>>();
cudaEventRecord(e1,stream1);
B 
C 
cudaEventRecord(e1,stream1);

cudaStreamWaitEvent(stream1,e2);
D 

//convert the stream to a graph
cudaStreamEndCapture(stream1,&graph);

2. Create Graphs Directly
    map Graph-Based Workflows Directly Into CUDA



In [ ]:
cudaGraphCreate(&graph);
cudaGraphAddNode(graph,kernel_a,{},...);
cudaGraphAddNode(graph,kernel_b,{kernel_a},...);

...

//Instantiate graph and apply optimizations
cudaGraphInstantiate(&instance, graph);

//Launch executable graph 100 times
for(int i=0;i<100;++i){
    cudaGraphLaunch(instance, stream);}
}

Graph Execution Semantics

    Order graph work with other non-graph CUDA work
    

In [ ]:
launchWork(cudaGraphExec_t i1, cudaGraphExec_t i2,
    CPU_FUNC cpu, cudaStream_t stream){

        A<<<256,256,0,stream>>>(); //kernel launch
        cudaGraphLaunch(i1,stream); //Graph launch
        cudaStreamAddCallback(stream,cpu);//CPU callback 
        cudaGraphLaunch(i2,stream); //Graph launch

        cudaStreamSynchronize(stream);



    }

Graphs ignore stream serialization rull

Launch Stream is Used *only* for ordering with other work

Graph is not automatic placement 

User must define Execution location for each *NODE*

Execution dependencies

Data dependency graph definition

Task Input Outpus
A nont X  




In [ ]:
cudaGraphAddNode(graph,A,{},...);

cudaGraphAddNode(graph,B,{A},...);

*ALL* **data dependencies** can trivially be mapped to **execution dependencies**, but
*NOT ALL* **execution dependencies** can be mapped to **data dependencies**.


**execution dependencies** is super sets.


#### What Graph can do

Rapid re_issue of work

for(int i=0;i<5;i++){
    launch_graph(G);
}

In [ ]:
# Graph e